## Descripccion
Este proyecto es la conbinacion de `CAMBIO_DE_MONEDA_Exchangeratesapi.io.ipynb` el cual se encuentra en la carpta de  <a href="https://github.com/Ncalderon1/IA/tree/main/Data_eng/Apis"> Apis </a>
  y de `WEB_scrapin_table.ipynb` dentro de la carpeta <a href="https://github.com/Ncalderon1/IA/tree/main/Data_eng/WEB_SCRAP"> WEB_scrap </a>. 

La finalidad es, mediante los archivos obtenidos en los dos proyectos "tasa de conversion de USD a otras monedas"`(Json)` y "Lista de los bancos más grandes organizados por capitalización de mercado"`(CSV)` realizar un proceso de `ETL`. Extraer los datos de el archivo Json ,transformar su columna de valor USD A GBP para finalmente ser cargado junto sus logs de informacion. 



In [2]:
import glob
import pandas as pd
from datetime import datetime

##  Extraer





Funcion extraer JSON 


In [4]:
def extract_from_json(file_to_process):
    dataframe = pd.read_json(file_to_process)
    return dataframe

Función de extracción

Extrae el archivo JSON `bank_market_cap_1.json` y  Almacena los datos en un data frame `pandas`. con un formato de columnas. 


In [6]:
def extract():
    
    extracted_data = pd.DataFrame(columns=['Name','Market Cap (US$ Billion)']) # crear un data frame vacío para contener los datos extraídos
   
    #procesar todos los archivos json 
    for jsonfile in glob.glob("*.json"):
        extracted_data = extracted_data.append(extract_from_json(jsonfile), ignore_index=True)
    
    return extracted_data

se carga el archivo `exchange_rates.csv` en un data frame para filtrar el tipo de cambio a libras esterlinas con el símbolo `GBP`.


In [7]:
df=pd.read_csv("exchange_rates.csv")
df.rename( columns={'Unnamed: 0':'exchange'}, inplace=True )
df1=df.loc[df["exchange"] == 'GBP']#solo libras esterlinas simbolo GBP
df1
exchange_rate=float(df1["Rates"].values)
print(exchange_rate)

0.7323984208000001


## Transformacion

Usando el valor anterior mente filtrado `usd a gbp` , se realiza el cambio de moneda.


In [8]:
def transform(data):
    data['Market Cap (US$ Billion)']= round(data['Market Cap (US$ Billion)']*exchange_rate,3)#transforma de USD a GBP y redondea a 3 decimales . 
    data=data.rename(columns={"Market Cap (US$ Billion)":"Market Cap (GBP$ Billion)"},inplace=False)#Renombra la columna "Market Cap (US$ Billion)" a "Market Cap (GBP$ Billion)".
    return data
    

## Load - Carga

carga de los datos en un csv denominado `bank_market_cap_gbp.csv`. 


In [9]:
targetfile="bank_matket_cap_gbp.csv"
def load(targetfile,data_to_load):
    data_to_load.to_csv(targetfile, index= False)  

## Registros


Funcion para registrar los datos en <code>log</code>:



In [10]:
def log(message):
    timestamp_format = '%Y-%h-%d-%H:%M:%S' # Año-Mes-Día-Hora-Minuto-Segundo
    now = datetime.now() # obtener la marca de tiempo actual
    timestamp = now.strftime(timestamp_format)
    with open("logfile.txt","a") as f:
        f.write(timestamp + ',' + message + '\n')
    

## Ejecución del proceso ETL

In [11]:
log("ETL Job Started")
log("Extract phase Started")


### Extract


In [12]:
#llamdo de la funcion
extracted_data = extract()
#imprimir
extracted_data.head(5)


,Name,Market Cap (US$ Billion)
0,JPMorgan Chase,390.934
1,Industrial and Commercial Bank of China,345.214
2,Bank of America,325.331
3,Wells Fargo,308.013
4,China Construction Bank,257.399


In [13]:
log("Extract phase Ended")


### Transform


In [14]:
log("Transform phase started")


In [15]:
# llamdo de la funcion
transformed_data=transform(extracted_data)
# impresion
transformed_data.head()


,Name,Market Cap (GBP$ Billion)
0,JPMorgan Chase,286.319
1,Industrial and Commercial Bank of China,252.834
2,Bank of America,238.272
3,Wells Fargo,225.588
4,China Construction Bank,188.519


In [16]:
log("Transform phase Ended")

### Load


In [17]:
log("Load phase Started")


In [18]:
load(targetfile,transformed_data)


In [19]:
log("Load phase Ended")
